<a href="https://colab.research.google.com/github/ashikita/qir-toolbox/blob/main/json2tsv-4qir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# input.txtを作成

!touch input.txt

In [ ]:
# -*- coding: utf-8 -*-
# input.txt を読み取り、xml:lang="" かつ要素本文が空でないときだけ
# xml:lang="en"（日本語が含まれれば ja）に書き換える。
# output.txt に書き出し、標準出力に完了メッセージを出す。

import re
from pathlib import Path

INPUT_FILE = "input.txt"
OUTPUT_FILE = "output.txt"

def contains_japanese(text: str) -> bool:
    jp_pattern = re.compile(r'[\u3040-\u30FF\u4E00-\u9FFF]')
    return bool(jp_pattern.search(text))

def replacer(match: re.Match) -> str:
    tag = match.group("tag")
    attrs = match.group("attrs")
    inner = match.group("inner")

    # 本文が空または空白だけなら xml:lang を変更しない
    if inner.strip() == "":
        return match.group(0)

    # 言語判定
    lang = "ja" if contains_japanese(inner) else "en"

    # xml:lang="" を xml:lang="xx" に置き換える
    new_attrs = re.sub(r'xml:lang=""', f'xml:lang="{lang}"', attrs)

    return f"<{tag}{new_attrs}>{inner}</{tag}>"

def main():
    text = Path(INPUT_FILE).read_text(encoding="utf-8")

    # 改良版正規表現：
    #   - 任意のタグ名
    #   - 任意の属性（xml:lang="" を内部に含む）
    #   - 任意の本文（非貪欲）
    pattern = re.compile(
        r'<(?P<tag>[A-Za-z0-9:_-]+)'
        r'(?P<attrs>[^>]*\s+xml:lang=""[^>]*)>'
        r'(?P<inner>.*?)'
        r'</(?P=tag)>',
        flags=re.DOTALL
    )

    # 安定化のため2〜3回実行
    for _ in range(3):
        new_text = pattern.sub(replacer, text)
        if new_text == text:
            break
        text = new_text

    Path(OUTPUT_FILE).write_text(text, encoding="utf-8")
    print(f"[Done] 修正を完了しました → '{OUTPUT_FILE}' に出力しました。")

if __name__ == "__main__":
    main()
